<a href="https://colab.research.google.com/github/ssroka/ML_with_IBM_GeoDN/blob/main/MPI_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://environmentalintelligencesuite.ibm.com

## Setup

In [ ]:
!pip install folium==0.2.1
!pip install ibmpairs
!pip install geopandas

!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install "basemap == 1.3.0b1" "basemap-data == 1.3.0b1"

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.0 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79809 sha256=da16e21127b39042febd890fc5665fb98c585652b7a6602eac9d613432858fa9
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 163 kB/s 
     |████████████████████████████████| 6.3 MB 39.4 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
libgeos-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/

Mounted at /content/drive


# import packages

In [ ]:
import ibmpairs
from ibmpairs import paw, authentication
import ibmpairs.authentication as authentication
import ibmpairs.client as client
import ibmpairs.query as query
import ibmpairs.catalog as catalog
import requests

import geopandas
import os
import PIL.Image
import pandas as pd
import numpy as np
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits

from datetime import datetime, timedelta
import calendar
from mpl_toolkits.basemap import Basemap
from csv import writer

## Credentials

In [ ]:
txt_file = '/content/drive/My Drive/Colab Notebooks/PAIRS_KEY.txt'
f = open(txt_file, "r")
PAIRS_API_KEY = f.read()
PAIRS_SERVER   = 'https://pairs.res.ibm.com'
PAIRS_CREDENTIALS = authentication.OAuth2(api_key = PAIRS_API_KEY)
# Best practice is not to include secrets in source code so we read
# a user name and password from operating system environment variables.
# You could set the user name and password in-line here but we don't
# recommend it for security reasons.
EIS_USERNAME='ssroka@mit.edu'
EIS_APIKEY=PAIRS_API_KEY

# Create an authentication object with credentials.
credentials  = authentication.OAuth2(username = EIS_USERNAME,
                                     api_key  = EIS_APIKEY)

# Add the credentials object to a client object.
eis_client = client.Client(authentication = credentials)

# initializing constants

In [ ]:
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

day = 1
hour = 0

met_vars = {'Lv' : 2260000.0,  # Latent heat of vaporization [J/K]
'cp' : 1000.0, # specific heat of air [J/(kg K)]
'a' : 17.27, # empirical constant for Td to q calc
'b' : 35.86 # empirical constant for Td to q calc
}

lat_lon_square = ["17", "-100", "32", "-78"]

## query generator

In [ ]:
def get_query(t_start,t_end,lat_lon_square,met_vars):
  print(t_start.strftime(iso8601))
  print(t_end.strftime(iso8601))
  query_json = {"layers": [
      {
          "aggregation": "Mean",
          "alias": "To",
          "id": "50055", # 100 hPa temp [K] ERA5 
          "output": "false",
          "dimensions" : [
            {"name" : "level", "value" : "100"}],
          "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
          ]}, 
          "type": "raster"
      },
      {
          "aggregation": "Mean",
          "alias": "p",
          "id": "49439", # surface pressure [Pa] ERA5 
          "output": "false",
          "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
          ]}, 
          "type": "raster"
      },
      {
          "aggregation": "Mean",
          "alias": "Ts",
          "id": "49442", # SST [K] ERA5
          "output": "false",
          "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
          ]}, 
          "type": "raster"
      },
      {
          "aggregation": "Mean",
          "alias": "Td",
          "id": "49422", # 2m dew pt [K] ERA5
          "output": "false",
          "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
          ]}, 
          "type": "raster"
      },
      {
          "aggregation": "Mean",
          "alias": "Ta",
          "id": "49423", # 2m temp [K] ERA5
          "output": "false",
          "temporal": {"intervals": [
              {"start": t_start.strftime(iso8601), "end": t_end.strftime(iso8601)}
          ]}, 
          "type": "raster"
       },
       {
          "alias": "e", # vapor pressure [Pa]
          "output": "false",
          "expression": "6.1078*math:exp({}*($Td-273.16)/($Td-{}))".format(met_vars['a'],met_vars['b'])
       },
      {
          "alias": "qa", # spc. humidity at Ta [kg/kg]
          "output": "false",
          "expression": "0.622*($e)/($p/100.0-0.378*($e))"
       },
      {
          "alias": "es", # saturation vapor pressure [Pa]
          "output": "false",
          "expression": "6.1078*math:exp({}*($Ts-273.16)/($Ts-{}))".format(met_vars['a'],met_vars['b'])
       },
      {
          "alias": "qs", # saturation spc. humidity at SST [kg/kg]
          "output": "false",
          "expression": "0.622*($es)/($p/100.0-0.378*($es))"
       },
      {
          "alias": "eps", # epsilon in Bister and Emanuel (1998)
          "expression": "($Ts - $To)/$To"
       },
      {
          "alias": "Dk", # specific enthalpy potential [J/kg] (some formullations use moist enthalpy potential, but dz = 2m here)
          "expression": "({}*($qs-$qa)+{}*($Ts-$Ta))".format(met_vars['Lv'],met_vars['cp'])
       },
      {
          "alias": "MPI", # maximum potential intensity [m/s]
          "output": "true",
          "expression": "math:sqrt(($Ts - $To)/$To*({}*($qs-$qa)+{}*($Ts-$Ta)))".format(met_vars['Lv'],met_vars['cp'])
      }],
      "name": "result",
      "spatial": {"coordinates": lat_lon_square, "type": "square"},
      "temporal": {"intervals": [
              {"snapshot": t_start.strftime(iso8601)}
      ]}
  }
  return query_json

## Loop Queries

In [ ]:
# initialize tiff lists
MPI_list = []
Dk_list = []
eps_list = []
mean_MPI_file = "/content/drive/My Drive/Colab Notebooks/MPI_trends/mean_MPI_year_month.csv"
day_start = 1

for year in range(2014,2015,1):
  print('{}==============================='.format(year))
  for month in range(1,13):
    timestamp_start = "{}-{}-{} {}:00".format(year,month,day_start,hour)
    print(timestamp_start)
    t_start = pd.Timestamp(timestamp_start)
    day_range = calendar.monthrange(year, month)
    timestamp_end = "{}-{}-{} {}:00".format(year,month,day_range[1],hour)
    t_end = pd.Timestamp(timestamp_end)
    query_json = get_query(t_start,t_end,lat_lon_square,met_vars)
    print(query_json)
    # Submit the query for raster --------------------------------------------------
    query_result = query.submit_check_status_and_download(query_json)
    # # Find layer files to load from downloaded zip.
    files = query_result.list_files()

    # Store file output in lists ---------------------------------------------------
    files = [x for x in files if not x.lower().endswith('json')]
    MPI_list.extend(x for x in files if 'MPI' in x)
    eps_list.extend(x for x in files if 'eps' in x)
    Dk_list.extend(x for x in files if 'Dk' in x)

    array_MPI = np.array(PIL.Image.open(MPI_list[-1]))
    plt.imshow(array_MPI, cmap = 'plasma',vmax=120,vmin=20,extent =[-100, -78,17, 32])
    plt.xlabel("longitude",fontsize=20)
    plt.ylabel("latitude",fontsize=20)
    plt.title("{}".format(year),fontsize=20)
    plt.colorbar()
    plt.savefig("/content/drive/My Drive/Colab Notebooks/MPI_trends/MPI_{}_{}".format(year,month))
    plt.close()

    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")

    array_MPI_nan = array_MPI.copy()
    array_MPI_nan[array_MPI_nan==-9999] = np.nan

    array_eps = np.array(PIL.Image.open(eps_list[-1]))
    array_eps_nan = array_eps.copy()
    array_eps_nan[array_eps_nan==-9999] = np.nan

    array_Dk = np.array(PIL.Image.open(Dk_list[-1]))
    array_Dk_nan = array_Dk.copy()
    array_Dk_nan[array_Dk_nan==-9999] = np.nan

    mean_MPI_year_month = [current_time,np.nanmean(array_MPI_nan),np.nanmean(array_eps_nan),np.nanmean(array_Dk_nan), year, month]
    # First, open the old CSV file in append mode, hence mentioned as 'a'
    # Then, for the CSV file, create a file object
    with open(mean_MPI_file, 'a', newline='') as f_object:  
      # Pass the CSV  file object to the writer() function
      writer_object = writer(f_object)
      # Result - a writer object
      # Pass the data in the list as an argument into the writerow() function
      writer_object.writerow(mean_MPI_year_month)  
      # Close the file object
      f_object.close()

2014===============================
2014-1-1 0:00
2014-01-01T00:00:00Z
2014-01-31T00:00:00Z
{'layers': [{'aggregation': 'Mean', 'alias': 'To', 'id': '50055', 'output': 'false', 'dimensions': [{'name': 'level', 'value': '100'}], 'temporal': {'intervals': [{'start': '2014-01-01T00:00:00Z', 'end': '2014-01-31T00:00:00Z'}]}, 'type': 'raster'}, {'aggregation': 'Mean', 'alias': 'p', 'id': '49439', 'output': 'false', 'temporal': {'intervals': [{'start': '2014-01-01T00:00:00Z', 'end': '2014-01-31T00:00:00Z'}]}, 'type': 'raster'}, {'aggregation': 'Mean', 'alias': 'Ts', 'id': '49442', 'output': 'false', 'temporal': {'intervals': [{'start': '2014-01-01T00:00:00Z', 'end': '2014-01-31T00:00:00Z'}]}, 'type': 'raster'}, {'aggregation': 'Mean', 'alias': 'Td', 'id': '49422', 'output': 'false', 'temporal': {'intervals': [{'start': '2014-01-01T00:00:00Z', 'end': '2014-01-31T00:00:00Z'}]}, 'type': 'raster'}, {'aggregation': 'Mean', 'alias': 'Ta', 'id': '49423', 'output': 'false', 'temporal': {'intervals':

Maximum Potential Intensity ($V_p$)

Defined as: The maximum azimuthal velocity a hurricane could theoretically achieve (Bister and Emanuel, 1998)

$V_p^2 = \frac{C_K}{C_D}\frac{T_s-T_o}{T_o}\Delta k$

where $\Delta k$ is the specific enthalpy potential between the saturated sea surface and that air above (taken to be at $z=2$m here). Specific enthalpy is $k=L_vq+c_pT$ where $L_v$ is the latent heat of vaporization, $q$ is the specific humidity, $c_p$ the specific heat of water, and $T$ is temperature. Here we let the ratio of exchange coefficients $C_K/C_D=1$.